# Projected Growth in Metropolitan Planning Organization

## Introduction

This script is meant to take MORPC County Fortcasts data to filter for Franklin and Delaware Counties from 2010-2021, and create a new combined table and line graph to visualize the employment levels in both counties.

### Process Outline

The process carried out by this workflow can be described as follows:
  - The script will retrieve the QCEW datasets from the US Bureau of Labor Statistics
  - Filter to only retain and sum records from Franklin or Delawre, Ohio 2010-2021
  - Identify and keep only the rows containing all industires and all coverage
  - Write aggregated table and line chart to Excel to visualize the changes in employment levels

## Setup

### Import required package

In [148]:
import csv
import os
import urllib.request
import numpy as np
import urllib
import matplotlib.pyplot as plt
import pandas as pd
import json
import xlsxwriter
from io import BytesIO
from tableschema import Table
import sys
sys.path.append(os.path.normpath("../morpc-common"))
import morpc

### Parameters

In [128]:
# Define input and output directories
INPUT_DIR = "./input_data"
OUTPUT_DIR = "./output_data"

### Define inputs

#### MORPC Data

In [129]:
COMBINED_TABLE_FILENAME = "All Data (Unformatted).csv"
COMBINED_TABLE_PATH = os.path.join(INPUT_DIR, COMBINED_TABLE_FILENAME)
COMBINED_TABLE_SCHEMA_FILENAME = COMBINED_TABLE_FILENAME.replace(".csv","_schema.json")
COMBINED_TABLE_SCHEMA_PATH = os.path.join(INPUT_DIR, COMBINED_TABLE_SCHEMA_FILENAME)
print("Data: {}".format(COMBINED_TABLE_PATH))
print("Schema: {}".format(COMBINED_TABLE_SCHEMA_PATH))

Data: ./input_data\All Data (Unformatted).csv
Schema: ./input_data\All Data (Unformatted)_schema.json


#### Complined Excel Sheet

In [130]:
EXCEL_TABLE_FILENAME = "compiled.xlsx"
EXCEL_TABLE_PATH = os.path.join(OUTPUT_DIR, EXCEL_TABLE_FILENAME)
print("Output data path: {}".format(EXCEL_TABLE_PATH))

Output data path: ./output_data\compiled.xlsx


### Define outputs

#### Franklin and Delaware counties employment level by year (2010-2021)

In [131]:
OUTPUT_TABLE_FILENAME = "combined.csv"
OUTPUT_TABLE_PATH = os.path.join(OUTPUT_DIR, OUTPUT_TABLE_FILENAME)
OUTPUT_TABLE_SCHEMA_FILENAME = OUTPUT_TABLE_FILENAME.replace(".csv","_schema.json")
OUTPUT_TABLE_SCHEMA_PATH = os.path.join(OUTPUT_DIR, OUTPUT_TABLE_SCHEMA_FILENAME)
print("Data: {}".format(OUTPUT_TABLE_PATH))
print("Schema: {}".format(OUTPUT_TABLE_SCHEMA_PATH))

Data: ./output_data\combined.csv
Schema: ./output_data\combined_schema.json


## Getting input data

In [132]:
# Path to the .xlsx file
excel_path = "../morpc-county-controls/deliverables/CountyControls_WEB.xlsx"

# Use ExcelFile to open the Excel file and iterate through sheets
with pd.ExcelFile(excel_path) as xls:
    for sheet_name in xls.sheet_names:
        # Read each sheet to a pandas DataFrame
        df = pd.read_excel(xls, sheet_name=sheet_name)
        
        # Generate CSV file name based on sheet name
        csv_file = f'./input_data/{sheet_name}.csv'
        
        # Save the DataFrame as a CSV file
        df.to_csv(csv_file, index=False)

        print(f'Saved {sheet_name} to {csv_file}')


Saved Sheet1 to ./input_data/Sheet1.csv


## Formatting and fitlering all data

In [133]:
# Read the CSV file into a DataFrame
df = pd.read_csv(COMBINED_TABLE_PATH)
df.head()

,COUNTY,CATEGORY,VARIABLE,2020,2021,2025,2030,2035,2040,2045,2050
0,15-County Region,Employment,Industrial Employment,271473.0,274455.0,286375,298045,314195,327874,340388,351959
1,10-County Region,Employment,Industrial Employment,247237.0,250283.0,262461,274305,290443,303964,316226,327549
2,Delaware,Employment,Industrial Employment,17983.0,18473.0,20432,22644,25026,27276,29472,31778
3,Fairfield,Employment,Industrial Employment,11631.0,11719.0,12071,12527,13191,13902,14625,15317
4,Fayette,Employment,Industrial Employment,4383.0,4394.0,4439,4520,4588,4659,4747,4851


### Filter for Franklin and Delaware

In [134]:
# Filter rows where COUNTY is "Delaware" or "Franklin"
filtered_df = df.loc[df['COUNTY'].isin(['Delaware', 'Franklin'])]

filtered_df.head()

,COUNTY,CATEGORY,VARIABLE,2020,2021,2025,2030,2035,2040,2045,2050
2,Delaware,Employment,Industrial Employment,17983.0,18473.0,20432,22644,25026,27276,29472,31778
5,Franklin,Employment,Industrial Employment,152138.0,153068.0,156790,161726,169623,176594,182552,187699
19,Delaware,Employment,Office Employment,45294.0,46527.0,51459,57034,63032,68700,74231,80036
22,Franklin,Employment,Office Employment,374910.0,377203.0,386374,398536,417998,435172,449857,462543
36,Delaware,Employment,Other Employment,11835.0,12156.0,13444,14900,16467,17948,19393,20909


#### Filter for Total Jobs

In [135]:
# Filter rows where CATEGORY is "Employment" AND VARIABLE is "Total Employment"
jobs_total_df = filtered_df.loc[(filtered_df['CATEGORY'] == 'Employment') & (filtered_df['VARIABLE'] == 'Total Employment')]

# Identify string and numerical columns
string_columns = jobs_total_df.select_dtypes(include=['object']).columns
numerical_columns = jobs_total_df.select_dtypes(include=[np.number]).columns

# Summing numerical columns
numerical_sum = jobs_total_df[numerical_columns].sum(axis=0)

# Convert numerical_sum to a DataFrame with a single row
jobs_total_df = pd.DataFrame([numerical_sum])

jobs_total_df['category'] = 'Jobs'

jobs_total_df.head()

,2020,2021,2025,2030,2035,2040,2045,2050,category
0,930781.0,938724.0,970499.0,1005943.0,1067270.0,1118372.0,1163678.0,1205236.0,Jobs


#### Filter for Total Population

In [136]:
# Filter rows where CATEGORY is "Population" AND VARIABLE is "Total Population"
population_total_df = filtered_df.loc[(filtered_df['CATEGORY'] == 'Population') & (filtered_df['VARIABLE'] == 'Population')]

# Identify string and numerical columns
string_columns = population_total_df.select_dtypes(include=['object']).columns
numerical_columns = population_total_df.select_dtypes(include=[np.number]).columns

# Summing numerical columns
numerical_sum = population_total_df[numerical_columns].sum(axis=0)

# Convert numerical_sum to a DataFrame with a single row
population_total_df = pd.DataFrame([numerical_sum])

population_total_df['category'] = 'Population'

population_total_df.head()

,2020,2021,2025,2030,2035,2040,2045,2050,category
0,1539075.0,1542154.0,1637143.0,1724574.0,1830707.0,1915468.0,1989539.0,2063611.0,Population


#### Filter for Total Households

In [137]:
# Filter rows where CATEGORY is "Households and Housing Units" AND VARIABLE is "Households"
households_total_df = filtered_df.loc[(filtered_df['CATEGORY'] == 'Households and Housing Units') & (filtered_df['VARIABLE'] == 'Households')]

# Identify string and numerical columns
string_columns = households_total_df.select_dtypes(include=['object']).columns
numerical_columns = households_total_df.select_dtypes(include=[np.number]).columns

# Summing numerical columns
numerical_sum = households_total_df[numerical_columns].sum(axis=0)

# Convert numerical_sum to a DataFrame with a single row
households_total_df = pd.DataFrame([numerical_sum])

# Add a new column 'category' and fill all cells with 'jobs'
households_total_df['category'] = 'Households'

households_total_df.head()

,2020,2021,2025,2030,2035,2040,2045,2050,category
0,601309.0,602250.0,638714.0,671962.0,712503.0,744711.0,772750.0,800791.0,Households


#### Combining and cleaning filtered data

In [155]:
stacked_df = pd.concat([jobs_total_df, population_total_df, households_total_df], axis=0)

# List of columns to keep
columns_to_keep = ['2020', '2050', 'category']

# Filter the DataFrame to keep only the specified columns
filtered_df = stacked_df[columns_to_keep]

# List of columns with "category" first
reordered_columns = ['category'] + [col for col in filtered_df.columns if col != 'category']

# Reorder the DataFrame
filtered_df = filtered_df[reordered_columns]

# Calculate the change for each category
filtered_df['change'] = filtered_df['2050'] - filtered_df['2020']

# List of columns to keep
columns_to_keep = [ 'category','2020', 'change',]

# Filter the DataFrame to keep only the specified columns
filtered_df = filtered_df[columns_to_keep]



# Define the desired order for the categories
category_order = ['Jobs','Households','Population']

# Reorder the DataFrame according to the specified category order
filtered_df['category'] = pd.Categorical(data['category'], categories=category_order, ordered=True)
filtered_df.sort_values('category', ascending=False, inplace=True)



filtered_df.head()
filtered_df.to_csv(OUTPUT_TABLE_PATH, index=False)

## Dataset and graph to Excel

In [164]:
# Load the data from CSV
data  = pd.read_csv(OUTPUT_TABLE_PATH)

# Create an Excel workbook and add a worksheet
workbook = xlsxwriter.Workbook(EXCEL_TABLE_PATH)
worksheet = workbook.add_worksheet()

# Write the column headers and the data to the worksheet
worksheet.write('A1', 'Category')
worksheet.write('B1', '2020')
worksheet.write('C1', 'Change')
worksheet.write_column('A2', data['category'])
worksheet.write_column('B2', data['2020'])
worksheet.write_column('C2', data['change'])

# Create a chart object
chart = workbook.add_chart({'type': 'column', 'subtype': 'stacked'})

# Configure the series for the '2020' data
chart.add_series({
    'name':       '2020',
    'categories': '=Sheet1!$A$2:$A$4',
    'values':     '=Sheet1!$B$2:$B$4',
    'fill':       {'color': morpc.CONST_MORPC_COLORS["midblue"]},
    'data_labels': {'value': True, 'font': {'color': 'white'}},
})

# Configure the series for the 'change' data
chart.add_series({
    'name':       'Change',
    'categories': '=Sheet1!$A$2:$A$4',
    'values':     '=Sheet1!$C$2:$C$4',
    'fill':       {'color': morpc.CONST_MORPC_COLORS["blue"]},
    'data_labels': {'value': True, 'font': {'color': 'white'}},
})

# Set the chart title and axis labels
chart.set_title({'name': 'FIGURE 2.1 PROJECTED GROWTH'})

# Customize the X-axis to make the line not visible
chart.set_x_axis({
    'line': {'none': True},
    'major_tick_mark': 'none',  # Ensure no tick marks
    'minor_tick_mark': 'none',
})

# Customize the Y-axis to make the line not visible
chart.set_y_axis({
    'line': {'color': '#D9D9D9', 'width': 1},  # Light grey line, adjust color as needed
    'major_gridlines': {'visible': True, 'line': {'color': '#D9D9D9', 'width': 0.5}},  # Light grey gridlines
    'minor_gridlines': {'visible': False},  # Typically, minor gridlines are kept invisible
    'major_tick_mark': 'none',  # Keep major tick marks invisible
    'minor_tick_mark': 'none',  # Keep minor tick marks invisible
})

# Remove the legend
chart.set_legend({'none': True})

# Set the chart style
chart.set_style(11)

# Insert the chart into the worksheet
worksheet.insert_chart('E2', chart)

# Close the workbook
workbook.close()

# The workbook is saved automatically when exiting the 'with' block
print("Excel file with bar charts created successfully.")

Excel file with bar charts created successfully.
